<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#获取基金费率等信息" data-toc-modified-id="获取基金费率等信息-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>获取基金费率等信息</a></span></li><li><span><a href="#获取基金经理信息" data-toc-modified-id="获取基金经理信息-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>获取基金经理信息</a></span></li><li><span><a href="#获取基金规模，年度收益信息" data-toc-modified-id="获取基金规模，年度收益信息-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>获取基金规模，年度收益信息</a></span></li><li><span><a href="#获取多只基金的信息" data-toc-modified-id="获取多只基金的信息-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>获取多只基金的信息</a></span></li><li><span><a href="#相关阅读" data-toc-modified-id="相关阅读-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>相关阅读</a></span></li></ul></div>

In [1]:
import pandas as pd
import datetime
import numpy as np
import tushare as ts

使用tushare 获取财经数据，首先需要注册，注册地址如下：

[点击注册tushare](https://tushare.pro/register?reg=129033)

然后按照下面的代码设置 token

以上方法只需要在第一次或者token失效后调用，完成调取tushare数据凭证的设置，正常情况下不需要重复设置。

In [86]:
# token='XXXXXXX'  
# ts.set_token(token)

pro = ts.pro_api()

## 获取基金费率等信息

In [75]:
# 获取基金中文名称信息
df_e = pro.fund_basic(market='E')
df_o = pro.fund_basic(market='O')
df_fund_info = df_e.append(df_o)
df_fund_info = df_fund_info[['ts_code','name', 'management','m_fee', 'c_fee','found_date']]
df_fund_info.columns = ['fund_code','fund_name', 'management','m_fee', 'c_fee','found_date']
df_fund_info = df_fund_info.reset_index(drop=True)
df_fund_info

,fund_code,fund_name,management,m_fee,c_fee,found_date
0,159713.SZ,稀土ETF,富国基金,0.50,0.10,20210805
1,561910.SH,电池ETF,招商基金,0.50,0.10,20210804
2,517010.SH,沪港深500ETF指数,易方达基金,0.15,0.05,20210803
3,159790.SZ,碳中和,华夏基金,0.50,0.10,20210730
4,159786.SZ,VRETF,银华基金,0.50,0.10,20210729
...,...,...,...,...,...,...
15862,516633.OF,华夏中证云计算与大数据主题ETF,华夏基金,0.50,0.10,None
15863,011481.OF,广发瑞锦一年定开,广发基金,1.00,0.10,None
15864,011749.OF,华泰柏瑞景气成长C,华泰柏瑞基金,1.50,0.25,None
15865,010721.OF,德邦安瑞C,德邦基金,0.70,0.10,None


## 获取基金经理信息

In [82]:
# 获取单个基金的信息

def get_fund(fund_code,df_fund_info):    
    # 获取基金经理信息，以及开始管理该基金的日期
    df_manager = pro.fund_manager(ts_code=fund_code)
    df_manager = df_manager[df_manager['end_date'].isna()]
    df_manager = df_manager.sort_values('begin_date',ascending=True).head(1)
    begin_date = df_manager['begin_date'].values[0] # 开始管理该基金的日期
    df_manager = df_manager[['ts_code', 'name', 'gender','begin_date']]
    df_manager.columns = ['fund_code', 'manager_name', 'gender','begin_date']
    
    # 合并
    df_fund = pd.merge(df_manager,df_fund_info,how='left',on='fund_code')

    return df_fund
    
# 110011.OF,易方达中小盘

df_fund = get_fund('110011.OF',df_fund_info) 
df_fund

,fund_code,manager_name,gender,begin_date,fund_name,management,m_fee,c_fee,found_date
0,110011.OF,张坤,M,20120928,易方达中小盘,易方达基金,1.5,0.25,20080619


## 获取基金规模，年度收益信息

In [83]:
# 获取单支基金的年度收益,基金规模等信息

def get_returns(fund_code,start_year):
    df_fund_nav = pro.fund_nav(ts_code=fund_code)
    df_fund_nav['date'] = pd.to_datetime(df_fund_nav['end_date'])
    df_fund_nav = df_fund_nav.set_index('date').sort_index()
    
    years = df_fund_nav['accum_nav'].resample('AS').sum().to_period('A')
    # 获取年度数据，类型为 pandas 的 period
    years = years.index.tolist()
    
    # 有些基金的开始日期晚于2018年，需要对齐列，补空白
    year_list = [yr.year for yr in years if yr.year>=start_year]
    
    # 将 period 时间数据转为 string 的年度数据
    years = [str(yr.year) for yr in years]

    df_yrs_index = pd.DataFrame()
    for yr in years:
        df_yr_index = df_fund_nav.loc[yr].tail(1)
        df_yrs_index = df_yrs_index.append(df_yr_index)
    df_yrs = df_fund_nav.head(1).append(df_yrs_index)

    # 计算每年的收益率，即涨跌幅度
    df_yrs['returns'] = df_yrs['accum_nav'].pct_change()

    # 删除收益率为 NA 的行 （第1天有数据记录的日期）
    df_yrs = df_yrs.dropna(subset=['returns'])

    # 筛选自开始年份以来的数据
    df_yrs = df_yrs.loc[str(start_year):]

    df_yrs = df_yrs.reset_index()
    df_yrs['year'] = df_yrs['date'].dt.year

    # 透视表
    df_yr_returns = pd.pivot_table(
        df_yrs, index=['ts_code'],
        values=['returns'], columns=['year'], fill_value=""
    )

    # 将多层索引转变为单层索引
    df_yr_returns = df_yr_returns['returns']
    df_yr_returns = df_yr_returns.reset_index()
    df_yr_returns.columns.name = None
    
    df_yr_returns = df_yr_returns.rename(columns={'ts_code':'fund_code'})
    
    # 基金份额
    df_fund_share = pro.fund_share(ts_code=fund_code).head(1)
    df_fund_share.columns = ['fund_code', 'fd_share_date', 'fd_share', 'fund_type', 'market']
    # fd_share,单位是 万份
    fd_share_date = df_fund_share['fd_share_date'].values[0] # 份额对应的日期
    
    # 份额日期的净值数据
    df_ann_nav = df_fund_nav.loc[fd_share_date:fd_share_date].sort_values('update_flag',ascending=False).head(1)
    df_ann_nav = df_ann_nav[['ts_code', 'accum_nav','unit_nav']]
    df_ann_nav.columns = ['fund_code', 'accum_nav','unit_nav']
    
    # 计算基金规模，amount，单位：亿元
    df_fund_amount = pd.merge(df_fund_share,df_ann_nav,how='left',on='fund_code')    
    df_fund_amount['amount'] = df_fund_amount['fd_share'] * df_fund_amount['unit_nav']/10000
    df_fund_amount = df_fund_amount[['fund_code','amount','fd_share_date']]
    
    # 合并数据    
    df_yr_returns = pd.merge(df_fund_amount,df_yr_returns,how='left',on='fund_code')

    for yr in year_list:
        if yr not in df_yr_returns.columns.tolist():
            df_yr_returns[yr]=np.nan
    
    return df_yr_returns

df_return = get_returns('110011.OF',2018)
df_return

,fund_code,amount,fd_share_date,2018,2019,2020,2021
0,110011.OF,287.010963,20210630,-0.129849,0.585684,0.715145,-0.089598


## 获取多只基金的信息

In [72]:
# 获取多只基金的信息

def get_data_fund(df_fund_info,fund_code_short,code_exception,start_year):
    df_filter_info = pd.DataFrame()
    for code in fund_code_short:
        df_tmp = df_fund_info[df_fund_info['fund_code'].str.contains(code)].head(1)
        df_filter_info = df_filter_info.append(df_tmp)
    df_filter_info = df_filter_info.reset_index(drop=True)   
    fund_codes = df_filter_info['fund_code'].tolist()
    # fund_codes

    df = pd.DataFrame()
    for code in fund_codes:
        if code in code_exception:
            code_update = code[:-2]+'OF'
            code_short = code[:-3]
            df_fund = get_fund(code_update,df_fund_info)
            df_fund['fund_code'] = code
            df_fund = df_fund[['fund_code','manager_name','gender','begin_date']]
            df_fund = df_fund.dropna(axis=1) # 删除含 NaN 的列
            test_m = df_fund_info[df_fund_info['fund_code'].str.contains(code_short)]
            df_fund = pd.merge(df_fund,test_m,how='left',on='fund_code')
        else:
            df_fund = get_fund(code,df_fund_info) 
        df_return = get_returns(code,start_year)
        df_fund_merge = pd.merge(df_fund,df_return,how='left',on='fund_code')
        df = df.append(df_fund_merge)
    df = df.reset_index(drop=True)
    
    df = df.rename(columns={
        'fund_code':'基金代码','manager_name':'基金经理',
        'gender':'性别','begin_date':'上任日期',
        'fund_name':'基金名称','management':'基金公司',
        'm_fee':'管理费','c_fee':'托管费',
        'found_date':'成立时间','amount':'基金规模(亿元)',
        'fd_share_date':'规模对应日期'
    })
    
    # 调整列的排序
    cols = df.columns.tolist()
    col_1 = cols[:4]
    col_2 = cols[4:5]
    col_3 = cols[5:]
    cols = col_2 + col_1 + col_3
    df = df[cols]

    return df

设置具体的参数

In [79]:
# 需要获取的基金代码简称列表
fund_code_short = ['000772', '003095','166005','320007',
                   '163406','260101','161005','162605',
                   '163402','005827','110022','110011',
                   '164908','000800','590008',
                   '360007','118001',
                   '519736','007119','002190','005911',
                   '001938','166002','377240'
                  ]

# ['200011','690003','200010',]

# 特列：LOF 基金，基金经理和管理费等费用所用的代码两处不一致，需要调整
code_exception = ['163406.SZ','160632.SZ','160222.SZ','159843.SZ','515920.SH'] 

# 设置基金收益开始的年份
start_year = 2018
# 设置基金收益截止的年份
end_year = 2021
year_list = list(range(start_year,end_year+1))

df_fund_final = get_data_fund(df_fund_info,fund_code_short,code_exception,start_year)
df_fund_final

,基金名称,基金代码,基金经理,性别,上任日期,基金公司,管理费,托管费,成立时间,基金规模(亿元),规模对应日期,2018,2019,2020,2021
0,景顺长城中国回报,000772.OF,韩文强,M,20191010,景顺长城基金,1.5,0.25,20141106,17.416544,20210630,-0.271923,0.388664,0.435860,-0.088832
1,中欧医疗健康A,003095.OF,葛兰,F,20160929,中欧基金,1.5,0.25,20160929,262.581451,20210630,-0.141935,0.699248,0.947456,0.032945
2,中欧价值发现A,166005.OF,曹名长,M,20151120,中欧基金,1.5,0.25,20090724,30.065539,20210630,-0.136789,0.160582,0.103410,0.080663
3,诺安成长,320007.OF,蔡嵩松,M,20190220,诺安基金,1.5,0.25,20090310,281.835362,20210630,-0.215530,0.561517,0.287915,0.291168
4,兴全合润,163406.SZ,谢治宇,M,20130129,兴证全球基金,1.5,0.25,20100422,320.540966,20210630,-0.255350,0.628653,0.751603,0.016275
5,景顺长城优选,260101.OF,杨锐文,M,20141025,景顺长城基金,1.5,0.25,20031024,59.458675,20210630,-0.105123,0.203162,0.283903,0.130095
6,富国天惠,161005.SZ,朱少醒,M,20051116,富国基金,1.5,0.25,20051116,408.520506,20210630,-0.113196,0.215553,0.256741,0.002101
7,景顺鼎益,162605.SZ,刘彦春,M,20150710,景顺长城基金,1.5,0.25,20050316,253.881272,20210630,-0.053947,0.195658,0.362284,-0.090734
8,兴全趋势,163402.SZ,董承非,M,20131028,兴证全球基金,1.5,0.25,20051103,384.198057,20210630,-0.064737,0.110949,0.140646,0.006767
9,易方达蓝筹精选,005827.OF,张坤,M,20180905,易方达基金,1.5,0.25,20180905,898.876822,20210630,-0.052900,0.551156,0.950922,-0.126409


In [80]:
def color_returns(val):
    if val >=0:
        color = '#EE7621' # light red 
    elif val <0:
        color = '#99ff66' # light green 
    else:
        color = '#FFFAFA' # ligth gray 
    return f'background-color: {color}'

format_dict = {'基金规模(亿元)': '￥{0:.1f}', 
               '管理费': '{0:.1f}', 
               '托管费': '{0:.2f}', 
               2017: '{0:.1%}', 
               2018: '{0:.1%}', 
               2019: '{0:.1%}', 
               2020: '{0:.1%}', 
               2021: '{0:.1%}', 
                }
df_fund_final = df_fund_final.sort_values('基金经理',ascending=True)

# df_fund_final.style.hide_index()\
#                     .hide_columns(['规模对应日期','基金公司','成立时间'])\
#                     .format(format_dict)\
#                     .background_gradient(subset=[2018],cmap='RdYlGn_r')\
#                     .background_gradient(subset=[2019],cmap='RdYlGn_r')\
#                     .background_gradient(subset=[2020],cmap='RdYlGn_r')\
#                     .background_gradient(subset=[2021],cmap='RdYlGn_r')\
#                     .background_gradient(subset=['基金规模(亿元)'],cmap='Blues')

df_fund_final.style.hide_index()\
                    .hide_columns(['规模对应日期','成立时间'])\
                    .format(format_dict)\
                    .applymap(color_returns,subset=year_list)\
                    .background_gradient(subset=['基金规模(亿元)'],cmap='Blues')

基金名称,基金代码,基金经理,性别,上任日期,基金公司,管理费,托管费,基金规模(亿元),2018,2019,2020,2021
睿远成长价值A,007119.OF,傅鹏博,M,20190326,睿远基金,1.5,0.15,￥330.0,nan%,20.8%,71.0%,3.4%
景顺鼎益,162605.SZ,刘彦春,M,20150710,景顺长城基金,1.5,0.25,￥253.9,-5.4%,19.6%,36.2%,-9.1%
广发双擎升级A,005911.OF,刘格菘,M,20181102,广发基金,1.5,0.25,￥167.2,-1.7%,121.7%,64.4%,9.1%
中欧时代先锋A,001938.OF,周应波,M,20151103,中欧基金,1.5,0.25,￥210.0,-8.3%,44.3%,37.5%,11.8%
中欧新蓝筹A,166002.OF,周蔚文,M,20110523,中欧基金,1.5,0.25,￥160.3,-8.6%,22.2%,20.9%,0.9%
易方达中小盘,110011.OF,张坤,M,20120928,易方达基金,1.5,0.25,￥287.0,-13.0%,58.6%,71.5%,-9.0%
易方达亚洲精选,118001.OF,张坤,M,20140408,易方达基金,1.5,0.35,￥41.9,-19.2%,23.9%,50.0%,-23.2%
易方达蓝筹精选,005827.OF,张坤,M,20180905,易方达基金,1.5,0.25,￥898.9,-5.3%,55.1%,95.1%,-12.6%
光大优势,360007.OF,戴奇雷,M,20191119,光大保德信基金,1.5,0.25,￥13.5,-21.6%,12.5%,10.7%,-1.0%
中欧价值发现A,166005.OF,曹名长,M,20151120,中欧基金,1.5,0.25,￥30.1,-13.7%,16.1%,10.3%,8.1%



## 相关阅读

- [视频：Plotly 和 Dash 在投资领域的应用](https://mp.weixin.qq.com/s/VopPrpe_64j5za4AYkmUTg)

- [视频：Plotly中绘制股票交易图表](https://mp.weixin.qq.com/s/1TUB2G-xavNm796uXUnRfQ)

- [Plotly中绘制三种经典的股票交易图表](https://mp.weixin.qq.com/s/1v3h5-5fNVk7cBdaRKzJkg)

- [神器Tushare，财经数据必备工具！](https://mp.weixin.qq.com/s/c1ukemeK12flCgA-lo69fA)

- [股票亏惨了，用Python来做一个投资计划](https://mp.weixin.qq.com/s/WYuMwCJBrWaBiDs8xp2KMA)

- [用 Python 读取巴菲特近期持仓数据](https://mp.weixin.qq.com/s/6h9Xq2lfrZSlRNw_W8aWcg)

更多内容，请关注公众号「**Python数据之道**」：

![](https://tva1.sinaimg.cn/large/008i3skNgy1gs2d9ca7dvj31hc0u07ng.jpg)